In [1]:
# In case: Not found: No algorithm worked!
# In case: This is probably because cuDNN failed to initialize

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np

In [3]:
path_saved = "../saved_model/"

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

In [5]:
base_dir = "../dataset/beach_mountain"
# base_dir = "drive/MyDrive/ML_colab/dataset/beach_mountain"
# base_dir = "drive/MyDrive/ML_colab/dataset/b_m/beach_mountain/"

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_beach_dir = os.path.join(train_dir, 'beach')
train_moutain_dir = os.path.join(train_dir, 'mountain')

validation_beach_dir = os.path.join(validation_dir, 'beach')
validation_mountain_dir = os.path.join(validation_dir, 'mountain')

test_beach_dir = os.path.join(test_dir, 'beach')
test_moutain_dir = os.path.join(test_dir, 'mountain')

In [6]:
# img_dim = (224,224)
img_dim = (150,150)

In [7]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG16

In [8]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))
#                   input_shape=(224, 224, 3))

# use pretrained places365
# local_weights_file = '../pretrained/vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5'
# conv_base = VGG16(weights=None,
#                   include_top=False,
#                   input_shape=(224, 224, 3))
# conv_base.load_weights(local_weights_file)

conv_base.trainable = False

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [9]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [11]:
batch_size = 40

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_dim,
    batch_size=batch_size,
    class_mode='binary') # because we use binary_crossentropy loss we need binary labels

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=img_dim,
    batch_size=batch_size,
    class_mode='binary')

Found 9000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [12]:
test_datagen = ImageDataGenerator( rescale = 1.0/255. )
validation_generator =  test_datagen.flow_from_directory( test_dir,
                                                          batch_size  = batch_size,
                                                          class_mode  = 'binary', 
                                                          target_size = img_dim)

Found 200 images belonging to 2 classes.


In [13]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

In [14]:
train_steps = train_generator.n//train_generator.batch_size
print("train steps: ", train_steps)

val_steps = validation_generator.n//validation_generator.batch_size
print("val steps: ", val_steps)

train steps:  225
val steps:  5


In [15]:
import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_acc') is not None and logs.get('val_acc') > 0.97) :
            print("\nReached 97.0% accuracy so cancelling training!")
            self.model.stop_training = True

my_callback = myCallback()

In [16]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=val_steps,
    verbose=2,
    callbacks=[my_callback])

Epoch 1/30


KeyboardInterrupt: 